In [1]:
import numpy as np
from PIL import Image
import tensorflow as tf
import os
from pickle import dump, load
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, load_model
import json
import requests

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/Source

In [8]:
def extract_features(filename, model):
    try:
        image = Image.open(filename)
    except:
        print("ERROR: Couldn't open image! Make sure the image path and extension is correct")
    image = image.resize((299,299))
    image = np.array(image)
    # for images that has 4 channels, we convert them into 3 channels
    if image.shape[2] == 4: 
        image = image[..., :3]
    image = np.expand_dims(image, axis=0)
    image = image/127.5
    image = image - 1.0
    feature = model.predict(image)
    print(image.shape," Shape of input to xception")
    return feature

In [9]:
tokenizer_filename = 'tokenizer.p'

In [10]:
def word_for_id(integer, tokenizer):
  for word, index in tokenizer.word_index.items():
      if index == integer:
          return word
  return None

In [11]:
def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'start'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)

        pred = model.predict([photo,sequence], verbose=0)

        pred = np.argmax(pred)
        
        word = word_for_id(pred, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'end':
            break
    return in_text

In [29]:
def test_img(img_path):
  max_length = 32
  tokenizer = load(open(tokenizer_filename,"rb"))

  model = load_model('model_19.h5')
  # xception_model = Xception(include_top=False, pooling="avg")
  xception_model=load_model('xception_model.h5')

  photo = extract_features(img_path, xception_model)
  img = Image.open(img_path)

  description = generate_desc(model,tokenizer,photo,max_length)
  print(description)
  plt.imshow(img)


In [30]:
dataset_images='Flicker8k_Dataset'

In [48]:
img = 'images/img8.jpg'
img_path = img

# print('Actual Caption:')
# for caption in descriptions[img]:
#   print(caption)

In [ ]:
print('Predicted Caption:')
test_img(img_path)